# Fermi-Hubbard Model

### Hamiltonian

$$
\begin{align}\begin{split}
    \hat{H} &= - J \sum_{\langle{i,\,j}\rangle}\sum_\sigma \hat{c}^{\dagger}_{i,\sigma} \hat{c}^{\phantom{\dagger}}_{i+1,\sigma} + U\sum_{i=1}^L  \hat{c}^{\dagger}_{i,\uparrow} \hat{c}^{\phantom{\dagger}}_{i,\uparrow}   \hat{c}^{\dagger}_{i,\downarrow} \hat{c}^{\phantom{\dagger}}_{i,\downarrow}, 
\end{split}\end{align}
$$

### Green functions

$$
    G^>_{ij}(t, t') = -i \left\langle \hat{c}^{\phantom{\dagger}}_{i,\uparrow}(t) \hat{c}^{{\dagger}}_{i,\uparrow}(t') \right\rangle\\
    F^>_{ij}(t, t') = -i \left\langle \hat{c}^{\phantom{\dagger}}_{i,\downarrow}(t) \hat{c}^{{\dagger}}_{i,\downarrow}(t') \right\rangle\\
$$

### Self-energies

Second-order Born approximation:
$$
    \Sigma_{ij, \uparrow}  (t, t') = U^2 G_{ij, \uparrow}(t, t') G_{ij, \downarrow}(t, t') G_{ji, \downarrow}(t', t),\\
    \Sigma_{ij, \downarrow}(t, t') = U^2 G_{ij, \downarrow}(t, t') G_{ij, \uparrow}(t, t') G_{ji, \uparrow}(t', t)
$$


$T$-matrix approximation:
$$
    \Sigma_{ij, \uparrow}  (t, t') = i U^2 T_{ij}(t, t') G_{ji, \downarrow}(t', t),\\
    \Sigma_{ij, \downarrow}(t, t') = i U^2 T_{ij}(t, t') G_{ji, \uparrow}(t', t)
$$

$$
   T_{ij}(t, t') =  \Phi_{ij}(t, t') - U \int_{\mathcal{C}}\mathrm{d}s\; \boldsymbol{\Phi}(t, s) \boldsymbol{T}(s, t')
$$

$$
    \Phi_{ij}(t, t') = -i G_{ij, \uparrow}(t, t') G_{ij, \downarrow}(t, t')
$$

$$
   T^\lessgtr_{ij}(t, t') =  \Phi^\lessgtr_{ij}(t, t') - U \int_{0}^{t}\mathrm{d}s\; [\boldsymbol{\Phi}^>(t, s) - \boldsymbol{\Phi}^<(t, s)] \boldsymbol{T}^\lessgtr(s, t') - U \int_{0}^{t'}\mathrm{d}s\; \boldsymbol{\Phi}^\lessgtr(t, s)[\boldsymbol{T}^<(s, t') - \boldsymbol{T}^>(s, t')]
$$

In [ ]:
using Pkg; Pkg.activate()

using KadanoffBaym, Interpolations, FFTW
using LinearAlgebra, BlockArrays


using JLD

using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")
using LaTeXStrings

## Helpers

In [ ]:
function fixed_point(F::Function, x0::AbstractArray; 
        mixing::Float64=0.5, 
        abstol::Float64=1e-12, 
        maxiter::Int=1000, 
        verbose::Bool=true, 
        norm=x -> LinearAlgebra.norm(x, Inf)
    )
    
    x_old = copy(x0)

    step = 0
    while step < maxiter
        x = F(x_old)
        res = norm(x - x_old)
        if verbose
            @info "step: $step // res: $res"
        end
        if res < abstol
            break
        end
        @. x_old = mixing * x + (1.0 - mixing) * x_old
        step += 1
    end

    if step == maxiter
        @warn "No convergence reached."
    end

    return x_old
end

## Solving

In [ ]:
Base.@kwdef struct FermiHubbardModel
    U::Float64
    
    # 8-site 3D cubic lattice
    H = begin
        h = BlockArray{ComplexF64}(undef_blocks, [4, 4], [4, 4])
        diag_block = [0 -1 0 -1; -1 0 -1 0; 0 -1 0 -1; -1 0 -1 0]
        setblock!(h, diag_block, 1, 1)
        setblock!(h, diag_block, 2, 2)
        setblock!(h, Diagonal(-1 .* ones(4)), 1, 2)
        setblock!(h, Diagonal(-1 .* ones(4)), 2, 1)

        full_h = BlockArray{ComplexF64}(undef_blocks, [8, 8], [8, 8])
        setblock!(full_h, h |> Array, 1, 1)
        setblock!(full_h, h |> Array, 2, 2)
        setblock!(full_h, zeros(ComplexF64, 8, 8), 1, 2)
        setblock!(full_h, zeros(ComplexF64, 8, 8), 2, 1)
        
        full_h |> Array
    end
    
    H1 = H[1:8, 1:8]
    H2 = H[1 + 8:2 * 8, 1 + 8:2 * 8]
end

In [ ]:
struct FermiHubbardData{T}
    GL::T
    GG::T
    FL::T
    FG::T
    
    TL::T
    TG::T    
    
    ΣNCA_c_L::T
    ΣNCA_c_G::T
    ΣNCA_f_L::T
    ΣNCA_f_G::T
    
    ΦL::T
    ΦG::T
    
    # Initialize problem
    function FermiHubbardData(GL::T, GG::T, FL::T, FG::T, TL::T, TG::T) where T
        new{T}(GL, GG, FL, FG, TL, TG, zero(GL), zero(GG), zero(FL), zero(FG), zero(FL), zero(FG))
    end
end

In [ ]:
begin
    function integrate1(hs::Vector, t1, t2, A::GreenFunction, B::GreenFunction, C::GreenFunction; tmax=t1)
        retval = zero(A[t1,t1])

        @inbounds for k in 1:tmax
            @views LinearAlgebra.mul!(retval, A[t1, k] - B[t1, k], C[k, t2], hs[k], 1.0)
        end
        return retval
    end

    function integrate2(hs::Vector, t1, t2, A::GreenFunction, B::GreenFunction, C::GreenFunction; tmax=t2)
        retval = zero(A[t1,t1])

        @inbounds for k in 1:tmax
            @views LinearAlgebra.mul!(retval, A[t1, k], B[k, t2] - C[k, t2], hs[k], 1.0)
        end
        return retval
    end
end;

In [ ]:
function fv!(model, data, out, times, h1, h2, t, t′)
    (; GL, GG, FL, FG, TL, TG, ΣNCA_c_L, ΣNCA_c_G, ΣNCA_f_L, ΣNCA_f_G) = data
    (; H1, H2, U) = model 
    
    # real-time collision integral
    ∫dt1(A,B,C) = integrate1(h1, t, t′, A, B, C)
    ∫dt2(A,B,C) = integrate2(h2, t, t′, A, B, C)
    
    ΣHF_c(t, t′) = 1.0im * U * Diagonal(FL[t, t])
    ΣHF_f(t, t′) = 1.0im * U * Diagonal(GL[t, t])
    
    out[1] = -1.0im * ((H1 + ΣHF_c(t, t′)) * GL[t, t′] + 
            ∫dt1(ΣNCA_c_G, ΣNCA_c_L, GL) + ∫dt2(ΣNCA_c_L, GL, GG)
        )

    out[2] = -1.0im * ((H1 + ΣHF_c(t, t′)) * GG[t, t′] + 
            ∫dt1(ΣNCA_c_G, ΣNCA_c_L, GG) + ∫dt2(ΣNCA_c_G, GL, GG)
        )

    out[3] = -1.0im * ((H2 + ΣHF_f(t, t′)) * FL[t, t′] + 
            ∫dt1(ΣNCA_f_G, ΣNCA_f_L, FL) + ∫dt2(ΣNCA_f_L, FL, FG)
        )

    out[4] = -1.0im * ((H2 + ΣHF_f(t, t′)) * FG[t, t′] +
            ∫dt1(ΣNCA_f_G, ΣNCA_f_L, FG) + ∫dt2(ΣNCA_f_G, FL, FG)
        )    
end

function fd!(model, data, out, times, h1, h2, t, t′)
    fv!(model, data, out, times, h1, h2, t, t)
    out[1:4] .-= adjoint.(out[1:4])
end

In [ ]:
function second_Born!(model, data, times, h1, h2, t, t′)
    (; GL, GG, FL, FG, TL, TG, ΣNCA_c_L, ΣNCA_c_G, ΣNCA_f_L, ΣNCA_f_G) = data
    (; U) = model
        
    if (n = size(GL, 3)) > size(ΣNCA_c_L, 3)
        resize!(ΣNCA_c_L, n)
        resize!(ΣNCA_c_G, n)
        resize!(ΣNCA_f_L, n)
        resize!(ΣNCA_f_G, n)
        
        resize!(TL, n)
        resize!(TG, n)
        resize!(data.ΦL, n)
        resize!(data.ΦG, n)
    end
    
    TL[t, t′] = -1.0im .* GL[t, t′] .* FL[t, t′]
    TG[t, t′] = -1.0im .* GG[t, t′] .* FG[t, t′]        
    
    ΣNCA_c_L[t, t′] = 1.0im .* U^2 .* TL[t, t′] .* transpose(FG[t′, t])
    ΣNCA_f_L[t, t′] = 1.0im .* U^2 .* TL[t, t′] .* transpose(GG[t′, t])
    
    ΣNCA_c_G[t, t′] = 1.0im .* U^2 .* TG[t, t′] .* transpose(FL[t′, t])
    ΣNCA_f_G[t, t′] = 1.0im .* U^2 .* TG[t, t′] .* transpose(GL[t′, t])
end

In [ ]:
function T_matrix!(model, data, times, h1, h2, t, t′)
    (; GL, GG, FL, FG, TL, TG, ΣNCA_c_L, ΣNCA_c_G, ΣNCA_f_L, ΣNCA_f_G, ΦL, ΦG) = data
    (; U) = model
        
    # real-time collision integral
    ∫dt1(A,B,C) = integrate1(h1, t, t′, A, B, C)
    ∫dt2(A,B,C) = integrate2(h2, t, t′, A, B, C)
    
    if (n = size(GL, 3)) > size(ΣNCA_c_L, 3)
        resize!(ΣNCA_c_L, n)
        resize!(ΣNCA_c_G, n)
        resize!(ΣNCA_f_L, n)
        resize!(ΣNCA_f_G, n)
        
        resize!(TL, n)
        resize!(TG, n)
        resize!(ΦL, n)
        resize!(ΦG, n)
    end
    
    # need to set all Φs at the very first t′ since they are _all_ known by then
    if t′ == 1
        for t′ in 1:t
            ΦL[t, t′] = -1.0im .* GL[t, t′] .* FL[t, t′]
            ΦG[t, t′] = -1.0im .* GG[t, t′] .* FG[t, t′]
        end
    end
    
    # cached version
    TL[t, t′], TG[t, t′] = let
        ∫dt_(x...) = (x[1] < 1 || x[2] < 1) ? zero(x[3][1,1]) : ∫dt(x...)
        
        ΦL_ = ΦL[t, t′]
        ΦG_ = ΦG[t, t′]
        
        I1 = integrate1(h1, t, t′, ΦG, ΦL, TL; tmax=t-1)
        I2 = integrate2(h2, t, t′, ΦL, TL, TG; tmax=t′-1)
        I3 = integrate1(h1, t, t′, ΦG, ΦL, TG; tmax=t-1)
        I4 = integrate2(h2, t, t′, ΦG, TL, TG; tmax=t′-1)
        
        L_ = ΦL[t, t′] - U * (I1 + I2)
        G_ = ΦG[t, t′] - U * (I3 + I4)
        
        fixed_point([L_, G_]; mixing=0.5, verbose=false) do x
            TL[t, t′], TG[t, t′] = x[1], x[2]

            [
                L_ - U * (h1[t] * (ΦG[t,t] - ΦL[t,t]) * TL[t,t′] + h2[t′] * ΦL[t,t′] * (TL[t′,t′] - TG[t′,t′])),
                G_ - U * (h1[t] * (ΦG[t,t] - ΦL[t,t]) * TG[t,t′] + h2[t′] * ΦG[t,t′] * (TL[t′,t′] - TG[t′,t′]))
            ]
        end
    end
    
    # uncached version
#     TL[t, t′], TG[t, t′] = fixed_point([ΦL[t, t′], ΦG[t, t′]]; mixing=0.5, verbose=false) do x
#         TL[t, t′], TG[t, t′] = x[1], x[2]
        
#         [
#             ΦL[t, t′] - U * (∫dt1(ΦG, ΦL, TL) + ∫dt2(ΦL, TL, TG)),
#             ΦG[t, t′] - U * (∫dt1(ΦG, ΦL, TG) + ∫dt2(ΦG, TL, TG))
#         ]
#     end
    
    ΣNCA_c_L[t, t′] = 1.0im .* U^2 .* TL[t, t′] .* transpose(FG[t′, t])
    ΣNCA_f_L[t, t′] = 1.0im .* U^2 .* TL[t, t′] .* transpose(GG[t′, t])
    
    ΣNCA_c_G[t, t′] = 1.0im .* U^2 .* TG[t, t′] .* transpose(FL[t′, t])
    ΣNCA_f_G[t, t′] = 1.0im .* U^2 .* TG[t, t′] .* transpose(GL[t′, t])
end

In [ ]:
# quantum numbers
dim = 8

# Define your Green functions at (t0, t0) – time arguments at the end
GL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
GG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
FL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
FG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)

TL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
TG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)

# Initial condition
N_c = zeros(dim)
N_f = zeros(dim)

N_c[1:4] = 0.1 .* [1, 1, 1, 1]
N_f[1:4] = 0.1 .* [1, 1, 1, 1]

N_c[5:8] = 0.0 .* [1, 1, 1, 1]
N_f[5:8] = 0.0 .* [1, 1, 1, 1]

GL[1, 1] = 1.0im * diagm(N_c)
GG[1, 1] = -1.0im * (I - diagm(N_c))
FL[1, 1] = 1.0im * diagm(N_f)
FG[1, 1] = -1.0im * (I - diagm(N_f))

TL[1, 1] = -1.0im .* GL[1, 1] .* FL[1, 1]
TG[1, 1] = -1.0im .* GG[1, 1] .* FG[1, 1]

data = FermiHubbardData(GL, GG, FL, FG, TL, TG)
data.ΦL[1, 1] = data.TL[1, 1]
data.ΦG[1, 1] = data.TG[1, 1]

data_2B = deepcopy(data)

model = FermiHubbardModel(U = 2.)

In [ ]:
tmax = 32.0;

# atol = 1e-10
# rtol = 1e-8;
atol = 1e-8
rtol = 1e-6;
# atol = 1e-6
# rtol = 1e-4;

In [ ]:
@time sol = kbsolve!(
    (x...) -> fv!(model, data, x...),
    (x...) -> fd!(model, data, x...),
    [data.GL, data.GG, data.FL, data.FG],
    (0.0, tmax);
    callback = (x...) -> T_matrix!(model, data, x...),
    atol = atol,
    rtol = rtol,
    dtini=1e-10,
#     qmax=4,
#     γ=19/20,
    stop = x -> (println("t: $(x[end])"); flush(stdout); false)
);

In [ ]:
save("FH_3D_T_matrix_sol_U_$(model.U)_tmax_$(tmax)_atol_$(atol)_rtol_$(rtol).jld", "solution", sol)

### Second Born approximation

In [ ]:
@time sol_2B = kbsolve!(
    (x...) -> fv!(model, data_2B, x...),
    (x...) -> fd!(model, data_2B, x...),
    [data_2B.GL, data_2B.GG, data_2B.FL, data_2B.FG],
    (0.0, tmax);
    callback = (x...) -> second_Born!(model, data_2B, x...),
    atol = atol,
    rtol = rtol,
    dtini=1e-10,
#     γ=19/20,
    stop = x -> (println("t: $(x[end])"); flush(stdout); false)
);

In [ ]:
save("FH_3D_second_Born_sol_U_$(model.U)_tmax_$(tmax)_atol_$(atol)_rtol_$(rtol).jld", "solution", sol_2B)

### Convergence

In [ ]:
U = 2.
tmax = 32.0;

In [ ]:
atol = 1e-6
rtol = 1e-4;

In [ ]:
loaded_sol_1 = load("FH_3D_T_matrix_sol_U_$(U)_tmax_$(tmax)_atol_$(atol)_rtol_$(rtol).jld")
GFs_1 = loaded_sol_1["solution"].u
num_points_1 = (loaded_sol_1["solution"].t |> size)[1]

In [ ]:
atol = 1e-8
rtol = 1e-6;

In [ ]:
loaded_sol_2 = load("FH_3D_T_matrix_sol_U_$(U)_tmax_$(tmax)_atol_$(atol)_rtol_$(rtol).jld")
GFs_2 = loaded_sol_2["solution"].u
num_points_2 = (loaded_sol_2["solution"].t |> size)[1]

In [ ]:
atol = 1e-10
rtol = 1e-8;

In [ ]:
loaded_sol_3 = load("FH_3D_T_matrix_sol_U_$(U)_tmax_$(tmax)_atol_$(atol)_rtol_$(rtol).jld")
GFs_3 = loaded_sol_3["solution"].u
num_points_3 = (loaded_sol_3["solution"].t |> size)[1]

In [ ]:
figure(figsize = (8, 3))

ax = subplot(121)
idx_1 = 1
ms = 4

plot([], [], label="\$\\texttt{rtol}\$", c="w")
plot(loaded_sol_3["solution"].t, [imag(GFs_3[1][idx_1, idx_1, k, k] .+ GFs_3[3][idx_1, idx_1, k, k]) for k = 1:num_points_3], 
    label = "\$ \\texttt{1e-8}\$", lw=1.5, "s", ms=ms, c = "C2")
plot(loaded_sol_2["solution"].t, [imag(GFs_2[1][idx_1, idx_1, k, k] .+ GFs_2[3][idx_1, idx_1, k, k]) for k = 1:num_points_2], 
    label = "\$ \\texttt{1e-6}\$", lw=1.5, "D", ms=ms, c = "C0")
plot(loaded_sol_1["solution"].t, [imag(GFs_1[1][idx_1, idx_1, k, k] .+ GFs_1[3][idx_1, idx_1, k, k]) for k = 1:num_points_1], 
    label = "\$ \\texttt{1e-4}\$", lw=1.5, "o", ms=ms, c = "C3")

plot(sol_2B.t, [imag(sol_2B.u[1][idx_1, idx_1, k, k] .+ sol_2B.u[3][idx_1, idx_1, k, k]) for k = 1:size(sol_2B.t)[1]], 
    lw=3, ls = "--", c = "k", alpha=0.75)

xlim(0, tmax)
ax.set_xticks(0:tmax / 4:tmax)
ylim(0, 0.3)
xlabel("\$J t\$")
ylabel("Charge on site 1", labelpad = 8)
ax.xaxis.set_tick_params(pad = xpad)
ax.yaxis.set_tick_params(pad = ypad)
ticklabel_format(axis = "y", style = "sci", scilimits = (-0, 0))
ax.legend(loc = "best", handlelength = 1, frameon = false, borderpad = 0, labelspacing = 0.25, ncol=2)

ax = subplot(122)
semilogy(loaded_sol_3["solution"].t, 
    [(sum(GFs_3[1][i, i, k, k] for i = 1:dim) |> imag) .+ (sum(GFs_3[3][i, i, k, k] for i = 1:dim) |> imag) for k = 1:num_points_3] 
    .- sum(N_c .+ N_f)  .|> abs, ls = "-", c = "C2")
semilogy(loaded_sol_2["solution"].t, 
    [(sum(GFs_2[1][i, i, k, k] for i = 1:dim) |> imag) .+ (sum(GFs_2[3][i, i, k, k] for i = 1:dim) |> imag) for k = 1:num_points_2] 
    .- sum(N_c .+ N_f)  .|> abs, ls = "-.", c = "C0")
semilogy(loaded_sol_1["solution"].t, 
    [(sum(GFs_1[1][i, i, k, k] for i = 1:dim) |> imag) .+ (sum(GFs_1[3][i, i, k, k] for i = 1:dim) |> imag) for k = 1:num_points_1] 
    .- sum(N_c .+ N_f)  .|> abs, ls = "--", c = "C3")
xlim(0, tmax)
ylim(1e-10, 2e-4)
ax.set_xticks(0:tmax / 4:tmax)
ylabel("\$Q(t) - Q_0\$", labelpad = 16)
xlabel("\$J t\$")
ax.xaxis.set_tick_params(pad = xpad)
ax.yaxis.set_tick_params(pad = ypad)
# ticklabel_format(axis = "y", style = "sci", scilimits = (-0, 0))
ax.yaxis.set_label_position("right")

tight_layout(pad = 0.1, w_pad = 0.5, h_pad = 0)
savefig("fermi_hubbard_T_matrix_convergence.pdf")